<h2>Imports</h2>

In [1]:
import os
import numpy as np
import torch
from collections import namedtuple
import csv
import re
from corpus import CornellMovieCorpus

/Users/darylramdin/opt/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h2>Use the GPU if present</h2>

In [2]:
#Let's do the GPU stuff
device = torch.device('mps')
if (torch.cuda.is_available()):
   device = torch.device('cuda')
print(device)

mps


<h2>Create a Cornell Movie Corpus </h2>

In [3]:
corpus = CornellMovieCorpus()

Loading movie lines...
['u0', 'u2', 'm0', "['L194', 'L195', 'L196', 'L197']"]
['u0', 'u2', 'm0', "['L198', 'L199']"]
['u0', 'u2', 'm0', "['L200', 'L201', 'L202', 'L203']"]
['u0', 'u2', 'm0', "['L204', 'L205', 'L206']"]
['u0', 'u2', 'm0', "['L207', 'L208']"]
['u0', 'u2', 'm0', "['L271', 'L272', 'L273', 'L274', 'L275']"]
['u0', 'u2', 'm0', "['L276', 'L277']"]
['u0', 'u2', 'm0', "['L280', 'L281']"]
['u0', 'u2', 'm0', "['L363', 'L364']"]
['u0', 'u2', 'm0', "['L365', 'L366']"]
Converting conversation line numbers to text...
Creating exchange pairs


<h2>Let's look at some data</h2>

In [4]:
lines = list(corpus.movie_lines.items())
for i in range(10):
    print(lines[i])

for i in range(10):
    print(corpus.movie_convos[i])

for i in range(10):
    print(corpus.movie_exchange_pairs[i])

('L1045', {'original_text': 'They do not!\n', 'prepped_text': 'they do not'})
('L1044', {'original_text': 'They do to!\n', 'prepped_text': 'they do to'})
('L985', {'original_text': 'I hope so.\n', 'prepped_text': 'i hope so'})
('L984', {'original_text': 'She okay?\n', 'prepped_text': 'she okay'})
('L925', {'original_text': "Let's go.\n", 'prepped_text': 'let s go'})
('L924', {'original_text': 'Wow\n', 'prepped_text': 'wow'})
('L872', {'original_text': "Okay -- you're gonna need to learn how to lie.\n", 'prepped_text': 'okay you re gonna need to learn how to lie'})
('L871', {'original_text': 'No\n', 'prepped_text': 'no'})
('L870', {'original_text': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n', 'prepped_text': 'i m kidding you know how sometimes you just become this persona and you don t know how to quit'})
('L869', {'original_text': 'Like my fear of wearing pastels?\n', 'prepped_text': 'like my fear of wearing pastels'})
['u

<h2>Create the vocabulary</h2>

In [5]:
corpus.create_vocabulary()

In [6]:
# # for i in range(len(cornell_corpus.vocabulary)):
# #     cornell_corpus.vocabulary[i] = (i,cornell_corpus.vocabulary[i])
# print(len(cornell_corpus.vocabulary),cornell_corpus.vocabulary[0:10])

In [8]:
print(corpus.vocabulary.vocabulary)

{'they': {'index': 0, 'count': 16996}, 'do': {'index': 1, 'count': 22791}, 'not': {'index': 2, 'count': 19555}, 'to': {'index': 3, 'count': 80773}, 'i': {'index': 4, 'count': 142514}, 'hope': {'index': 5, 'count': 1023}, 'so': {'index': 6, 'count': 13518}, 'she': {'index': 7, 'count': 12255}, 'okay': {'index': 8, 'count': 4620}, 'let': {'index': 9, 'count': 6187}, 's': {'index': 10, 'count': 67484}, 'go': {'index': 11, 'count': 9971}, 'wow': {'index': 12, 'count': 268}, 'you': {'index': 13, 'count': 148956}, 're': {'index': 14, 'count': 22084}, 'gonna': {'index': 15, 'count': 4319}, 'need': {'index': 16, 'count': 3995}, 'learn': {'index': 17, 'count': 428}, 'how': {'index': 18, 'count': 11446}, 'lie': {'index': 19, 'count': 461}, 'no': {'index': 20, 'count': 19583}, 'm': {'index': 21, 'count': 23167}, 'kidding': {'index': 22, 'count': 403}, 'know': {'index': 23, 'count': 21928}, 'sometimes': {'index': 24, 'count': 724}, 'just': {'index': 25, 'count': 15878}, 'become': {'index': 26, 'co